In [1]:
library(caret)
library(tidyverse)
library(tictoc)
library(doParallel)
library(parallel)
setwd('/media/jochum00/Aagaard_Raid3/michael_files/cr/')

Loading required package: lattice

Loading required package: ggplot2

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0
✔ purrr   0.3.4     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ purrr::lift()   masks caret::lift()

Loading required package: foreach


Attaching package: ‘foreach’


The following objects are masked from ‘package:purrr’:

    accumulate, when


Loading required package: iterators

Loading required package: parallel



# 0. import and preprocess the data

In [2]:
df<-as_tibble(read.table("crypto_sheet_2020-12-08_v3.tsv",sep = "\t",quote = "",header = T,row.names = NULL))
df$id<-factor(df$id)
df$symbol<-NULL
df$ranknow<-as_factor(df$ranknow)
vars<-colnames(df)

In [3]:
df_full<-df
length(unique(df$slug))

[1] 2073

In [4]:
df<-df_full %>%select(-c(high,low,close,volume, market,spread,net,name))
df<-df[complete.cases(df), ]
df<-df%>%arrange(ranknow)
df$date<-as.Date(df$date)
df_dec<-df%>%filter(date>"2020-12-01")
df_dec$slug<-factor(df_dec$slug)
df<-df%>%filter(slug %in% df_dec$slug)
df$slug<-factor(df$slug)
df$ranknow<-as.numeric(df$ranknow)
df$id<-as.factor(df$id)
df$day<-factor(df$day)
df$month<-factor(df$month)
df$year<-factor(df$year)
df<-df %>%filter(netnorm<200)%>%filter(netnorm>-50)
df<-df%>%filter(date>"2018-01-01")
df$slug<-factor(df$slug)

In [5]:
nrows<-nrow(df)
testnrows<-nrow(df)*0.2
test<-df[sample(nrow(df), testnrows), ]
train<-df%>%filter(!id %in%test$id)
nrows<-nrow(train)
trainrows<-nrow(train)*0.2
test_small<-train[sample(nrow(train),trainrows), ]
train_small<-train%>%filter(!id %in%test_small$id)
train$id<-NULL
test$id<-NULL
train_small$id<-NULL
test_small$id<-NULL

# 1. modeling
start up the cluster

In [6]:
cl <- makePSOCKcluster(48)
registerDoParallel(cl)

make 3 functions that will:
- run a model
- make predictions
- evaluate the model

In [7]:
tryme<-function(meth){
    tic()
    mod<-train(netnorm ~ .,data = train,method = meth)
    toc()
    return(mod)
}
predictme<-function (model){
    model_pred <- predict(model, test)
    return(model_pred)
}
evaluate<-function (model_pred){
    model_pred <- predict(model, test)
    postres<-postResample(pred = model_pred, obs = test$netnorm)
    return(postres)
}

# Evaluate the most disparate 4 models based on Jaccard difference

https://topepo.github.io/caret/models-clustered-by-tag-similarity.html

## [1] "Support Vector Machines with Radial Basis Function Kernel (svmRadial)"

In [8]:
mod_svmRadial<-tryme('svmRadial')

Warning message in .local(x, ...):
“Variable(s) `' constant. Cannot scale data.”
Warning message in .local(x, ...):
“Variable(s) `' constant. Cannot scale data.”


8426.345 sec elapsed


In [9]:
saveRDS(object = mod_svmRadial, file = "mod_svmRadial.rds")

In [10]:
install.packages('earth')

also installing the dependencies ‘plotrix’, ‘plotmo’, ‘TeachingDemos’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [ ]:
mod_bagEarth<-tryme('bagEarth')

In [17]:
saveRDS(object = mod_bagEarth, file = "mod_bagEarth.RDS")

ERROR: Error in saveRDS(object = mod_bagEarth, file = "mod_bagEarth.RDS"): object 'mod_bagEarth' not found


In [ ]:
mod_bam <- train(netnorm~.,
                   data = train,
                   method = "bam")
                   trControl = trainControl(method = "cv"))

In [18]:
saveRDS(object = mod_bam, file = "mod_bam.rds")

ERROR: Error in saveRDS(object = mod_bam, file = "mod_bam.rds"): object 'mod_bam' not found


In [16]:
ls()
getwd()


[1] "cl"            "df"            "df_dec"        "df_full"      
 [5] "evaluate"      "mod_svmRadial" "nrows"         "predictme"    
 [9] "test"          "test_small"    "testnrows"     "train"        
[13] "train_small"   "trainrows"     "tryme"         "vars"

[1] "/media/jochum00/Aagaard_Raid3/michael_files/cr"